# Assignment 4

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

### Exercise 1 (Performing PCA)

In [ ]:
#calculate the eigenvalue and eigenvector
def pca(data):
    cov_matrix = np.cov(data.T) # the definition of covariance matrix
    evals, evecs = np.linalg.eigh(cov_matrix)
    evals = evals[::-1]
    evecs = evecs[:,::-1]
    return evals, evecs

# def standardize(data):
#     data = center(data)
#     data = normalize(data)
#     return data

# def center(data):
#     mean = np.mean(data, axis = 0)
#     data = data - mean
#     return data

# def normalize(data):
#     sd = np.std(data, axis = 0)
#     data = data/sd
#     return data

In [ ]:
OccupancyTrain = np.loadtxt('OccupancyTrain.csv', delimiter = ',')
data_occ = OccupancyTrain[:,:-1] #except lables
#data_occ  = standardize(data_occ)
eigenvalues,eigenvectors = pca(data_occ)
# variance = eigenvalues/sum(eigenvalues) #calculate the proportion
variance = eigenvalues
count = list(range(1, len(variance)+1))

fig = plt.figure(figsize=(8,4))
plt.plot(count, variance)
plt.title('Variance versus Principcal Component Index of Occupancy data')
plt.xlabel('Index of Principcal Component')
plt.ylabel('Variance')
plt.xticks(count)

plt.savefig("ex1_f1.png", dpi = 200)

In [ ]:
IDSWeedCropTrain = np.loadtxt('IDSWeedCropTrain.csv', delimiter = ',')
data_pes = IDSWeedCropTrain[:,:-1] #except lables
#data_pes  = standardize(data_pes)
eigenvalues,eigenvectors = pca(data_pes)
# variance = eigenvalues/sum(eigenvalues) #the proportion of variance
variance = eigenvalues

count = list(range(1, len(variance)+1))
cum_variance = np.cumsum(eigenvalues/sum(eigenvalues))

fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(2,2,1,title='Variance versus PC Index of Pesticide data',\
                      xlabel='Index of Principcal Component',ylabel='Variance')
ax1 = plt.plot(count, variance)
ax1 = plt.xticks(count)
ax2 = fig.add_subplot(2,2,2,title='Cumulative Variance versus PC Index of Pesticide data',\
                      xlabel='Index of Principcal Component',ylabel='Proportion of Cumulative Variance')
ax2 = plt.plot(count, cum_variance)
ax2 = plt.xticks(count)
fig.subplots_adjust(wspace=1)
fig.tight_layout(pad=5)

plt.savefig("ex1_f2.png", dpi = 200)

In [ ]:
def needed_PCs_90(cum_variance):
    j = 0
    for i in cum_variance:
        j = j + 1
        if i > 0.90:
            return("the needed number of PCs to capture 90% of the variance is "+str(j))
            break
        else:
            continue

def needed_PCs_95(cum_variance):
    j = 0
    for i in cum_variance:
        j = j + 1
        if i > 0.95:
            return("the needed number of PCs to capture 95% of the variance is "+str(j))
            break
        else:
            continue

print(needed_PCs_90(cum_variance))
print(needed_PCs_95(cum_variance))  

### Exercise 2 (Visualization in 2D)

In [ ]:
# def pca_trans(data,n):
#     cov_matrix = np.cov(data.T) # the definition of covariance matrix
#     evals, evecs = np.linalg.eigh(cov_matrix)
#     evals = evals[::-1]
#     evecs = evecs[:,::-1]
#     evecs = evecs[:,:n]
#     data_tran = evecs.T @ data.T
#     data_tran = data_tran.T
#     return data_tran

def pca_trans(data,n):
    modelPCA2 = PCA(n_components=n)
    Xtrans = modelPCA2.fit_transform(data)
    return Xtrans

In [ ]:
tran_occ = pca_trans(data_occ,2)
plt.scatter(tran_occ[:,0],tran_occ[:,1],color='blue',s=3)
plt.title('Principal Component Analysis of Occupancy data')
plt.xlabel('Principcal Component 1')
plt.ylabel('Principcal Component 2')
plt.savefig("ex2_f1.png", dpi = 200)

In [ ]:
tran_pes = pca_trans(data_pes,2)
plt.scatter(tran_pes[:,0],tran_pes[:,1],color='blue',s=3)
plt.title('Principal Component Analysis of Pesticide data')
plt.xlabel('Principcal Component 1')
plt.ylabel('Principcal Component 2')
plt.savefig("ex2_f2.png", dpi = 200)

### Exercise 4 (Clustering II)

In [ ]:
cc = pd.DataFrame()
cc1 = pd.DataFrame([5.69, 49.38, 791.59, 3847.71, 3385.89, 1359.89, 293.73, 131.61, 70.73, 39.64, 19.44, 4.24, 0.44]).T
cc2 = pd.DataFrame([2.19, 13.73, 170.94, 1394.36, 3188.53, 2624.62, 1003.72, 632.81, 495.83, 295.4, 145.8, 29.2, 2.84]).T
cc = pd.concat([cc1,cc2],axis=0)
tran_cc = pca_trans(cc,2)
tran_cc

In [ ]:
tran_pes = pd.DataFrame(tran_pes)
df = pd.DataFrame()
df["PC1"] = tran_pes[0]
df["PC2"] = tran_pes[1]
df["label"] = pd.DataFrame(IDSWeedCropTrain[:, -1])

type0 = df[df['label'].isin([0])]
type1 = df[df['label'].isin([1])]

df_type0 = pd.DataFrame()
df_type0["PC1"] = type0["PC1"]
df_type0["PC2"] = type0["PC2"]
df_type0["label"]= type0["label"]

df_type1 = pd.DataFrame()
df_type1["PC1"] = type1["PC1"]
df_type1["PC2"] = type1["PC2"]
df_type1["label"]= type1["label"]

In [ ]:
fig = plt.figure(figsize=(7,4))
plt.scatter(data=type0,x='PC1',y='PC2',color='mediumturquoise',s=3,label='weed')
plt.scatter(data=type1,x='PC1',y='PC2',color='crimson',s=3,label='crop')
plt.scatter(x=1505.15, y=0,s=25,marker='x',c='black',label='Centroid 1')
plt.scatter(x=-1505.15, y=0,s=15,marker='D',c='black',label='Centroid 2')
plt.legend(loc='lower right')

plt.title('Principal Component Analysis of Pesticide data')
plt.xlabel('Principcal Component 1')
plt.ylabel('Principcal Component 2')
plt.savefig("ex4_f1.png", dpi = 200)

### Exercise 5 (Clustering III)

In [ ]:
# # it could work on Colab
# startingPoint = np.vstack((data_occ[0,], data_occ[1,]))
# kmeans_occ = KMeans(2, n_init = 1, init= startingPoint, algorithm = "full").fit(data_occ) #set k=2
# print(kmeans_occ.cluster_centers_)

Centroid 1: [2.03243556e+01 2.46537811e+01 5.75966874e+01 4.74787474e+02 3.62013593e-03]

Centroid 2: [2.18266607e+01 2.73984652e+01 4.58024161e+02 1.08964275e+03 4.43437128e-03]

In [ ]:
cc_occ = pd.DataFrame()
cc1_occ = pd.DataFrame([20.32, 24.65, 57.6, 474.79, 0.0036]).T
cc2_occ = pd.DataFrame([21.83, 27.4, 458.02, 1089.64, 0.0044]).T
cc_occ = pd.concat([cc1_occ,cc2_occ],axis=0)
tran_cc_occ = pca_trans(cc_occ,2)
print(tran_cc_occ)

In [ ]:
tran_occ = pd.DataFrame(tran_occ)
df = pd.DataFrame()
df["PC1"] = tran_occ[0]
df["PC2"] = tran_occ[1]
df["label"] = pd.DataFrame(OccupancyTrain[:, -1])

type0 = df[df['label'].isin([0])]
type1 = df[df['label'].isin([1])]

df_type0 = pd.DataFrame()
df_type0["PC1"] = type0["PC1"]
df_type0["PC2"] = type0["PC2"]
df_type0["label"]= type0["label"]

df_type1 = pd.DataFrame()
df_type1["PC1"] = type1["PC1"]
df_type1["PC2"] = type1["PC2"]
df_type1["label"]= type1["label"]

In [ ]:
fig = plt.figure(figsize=(7,4))
plt.scatter(data=type0,x='PC1',y='PC2',color='mediumturquoise',s=3,label='not occupied')
plt.scatter(data=type1,x='PC1',y='PC2',color='crimson',s=3,label='occupied')
plt.scatter(x=366.87, y=0,s=25,marker='x',c='black',label='Centroid 1')
plt.scatter(x=-366.87, y=0,s=15,marker='D',c='black',label='Centroid 2')
plt.legend(loc='lower right')

plt.title('Principal Component Analysis of Occupancy data')
plt.xlabel('Principcal Component 1')
plt.ylabel('Principcal Component 2')
plt.savefig("ex5_f1.png", dpi = 200)

### Exercise 6 (Linear Regression)

In [ ]:
def multivarlinreg(X, t):
    row = len(X)
    col_1 = np.ones(row)
    X = np.c_[col_1, X] # add a column of ones at the first column    
    w = np.linalg.inv(X.T @ X) @ X.T @ t
    return w

In [ ]:
OccupancyTrain = np.loadtxt('OccupancyTrain.csv', delimiter = ',')

X_Train = OccupancyTrain[:, :-1]
# X_Train = standardize(data=X_Train) #use Train data to standardize
X_Train_col5 = X_Train[:,-1] #only the humidity feature
Y_Train = OccupancyTrain[:, -1:] #the label

weights_f5 = multivarlinreg(X=X_Train_col5, t=Y_Train) #only use the humidity feature to perform regression
print(weights_f5)
print()

weights_all = multivarlinreg(X=X_Train, t=Y_Train) #only all feature to perform regression
print(weights_all)

### Exercise 7 (Evaluating Linear Regression)

In [ ]:
def rmse(prediciton, true):
    error = (true - prediction)**2
    rmse = np.sqrt(np.mean(error))
    return rmse

def test(weights, test_x, test_y):
    row = len(test_x)
    col_1 = np.ones(row)
    X = np.c_[col_1, test_x] # add a column of ones at the first column    
    prediction = X @ weights
    error = (test_y - prediction)**2
    rmse = np.sqrt(np.mean(error))
    return rmse

In [ ]:
OccupancyTest = np.loadtxt('OccupancyTest.csv', delimiter = ',')

X_Test = OccupancyTest[:, :-1]
# X_Test = standardize(data=X_Test)  #use Train data to standardize
X_Test_col5 = X_Test[:,-1]
Y_Test = OccupancyTest[:, -1:]

weights_f5_test = multivarlinreg(X=X_Train_col5, t=Y_Train) #only use the humidity feature to perform regression
weights_all_test = multivarlinreg(X=X_Train, t=Y_Train) #only all feature to perform regression

rmse_five = test(weights_f5_test, X_Test_col5, Y_Test)
rmse_all = test(weights_all_test, X_Test, Y_Test)
print(rmse_five)
print(rmse_all)